In [1]:
import wandb
import pandas as pd
from fastai.vision.all import *
from fastai.callback.wandb import WandbCallback

import params
from utils import get_predictions, create_iou_table, MIOU, BackgroundIOU, \
                  RoadIOU, TrafficLightIOU, TrafficSignIOU, PersonIOU, VehicleIOU, BicycleIOU

In [2]:
set_seed(42, reproducible=True)

In [3]:
train_config = SimpleNamespace(
    framework="fastai",
    img_size=(180, 320),
    batch_size=8,
    augment=True, # use data augmentation
    epochs=10, 
    lr=2e-3,
    pretrained=True  # whether to use pretrained encoder,
)

In [4]:
run = wandb.init(project=params.WANDB_PROJECT, entity=params.ENTITY, job_type="training", config=train_config)

In [5]:
processed_data_at = run.use_artifact(f'{params.PROCESSED_DATA_AT}:latest')
processed_dataset_dir = Path(processed_data_at.download())
df = pd.read_csv(processed_dataset_dir / 'data_split.csv')

In [6]:
df = df[df.Stage != 'test'].reset_index(drop=True)
df['is_valid'] = df.Stage == 'valid'

In [7]:
def label_func(fname):
    return (fname.parent.parent/"labels")/f"{fname.stem}_mask.png"

In [8]:
# assign paths
df["image_fname"] = [processed_dataset_dir/f'images/{f}.jpg' for f in df.File_Name.values]
df["label_fname"] = [label_func(f) for f in df.image_fname.values]

In [9]:
def get_data(df, bs=4, img_size=(180, 320), augment=True):
    block = DataBlock(blocks=(ImageBlock, MaskBlock(codes=params.BDD_CLASSES)),
                  get_x=ColReader("image_fname"),
                  get_y=ColReader("label_fname"),
                  splitter=ColSplitter(),
                  item_tfms=Resize(img_size),
                  batch_tfms=aug_transforms() if augment else None,
                 )
    return block.dataloaders(df, bs=bs)

In [10]:
config = wandb.config

In [11]:
dls = get_data(df, bs=config.batch_size, img_size=config.img_size, augment=config.augment)

In [12]:
metrics = [MIOU(), BackgroundIOU(), RoadIOU(), TrafficLightIOU(), \
           TrafficSignIOU(), PersonIOU(), VehicleIOU(), BicycleIOU()]

learn = unet_learner(dls, arch=resnet18, pretrained=config.pretrained, metrics=metrics)

In [13]:
callbacks = [
    SaveModelCallback(monitor='miou')
    WandbCallback(log_preds=False, log_model=True)
]

In [14]:
callbacks = [
    SaveModelCallback(monitor='miou'),
    WandbCallback(log_preds=False, log_model=True)
]

In [15]:
learn.fit_one_cycle(config.epochs, config.lr, cbs=callbacks)

epoch,train_loss,valid_loss,miou,background_iou,road_iou,traffic_light_iou,traffic_sign_iou,person_iou,vehicle_iou,bicycle_iou,time


epoch,train_loss,valid_loss,miou,background_iou,road_iou,traffic_light_iou,traffic_sign_iou,person_iou,vehicle_iou,bicycle_iou,time


epoch,train_loss,valid_loss,miou,background_iou,road_iou,traffic_light_iou,traffic_sign_iou,person_iou,vehicle_iou,bicycle_iou,time


epoch,train_loss,valid_loss,miou,background_iou,road_iou,traffic_light_iou,traffic_sign_iou,person_iou,vehicle_iou,bicycle_iou,time


epoch,train_loss,valid_loss,miou,background_iou,road_iou,traffic_light_iou,traffic_sign_iou,person_iou,vehicle_iou,bicycle_iou,time


...

epoch,train_loss,valid_loss,miou,background_iou,road_iou,traffic_light_iou,traffic_sign_iou,person_iou,vehicle_iou,bicycle_iou,time
0,0.505617,0.387823,0.303909,0.832589,0.692566,0.000000,0.000000,0.000000,0.602210,0.000000,00:51
1,0.419773,0.369582,0.311889,0.865683,0.726510,0.000000,0.000000,0.000000,0.591029,0.000000,00:51
2,0.348876,0.284566,0.337992,0.897110,0.801128,0.000000,0.000000,0.000000,0.667709,0.000000,00:52
3,0.296886,0.258616,0.346980,0.907943,0.815721,0.000000,0.000000,0.000000,0.705198,0.000000,00:52
4,0.282345,0.303062,0.344276,0.902011,0.818458,0.000000,0.000000,0.000000,0.689461,0.000000,00:53
5,0.248362,0.232243,0.354253,0.913885,0.831946,0.020242,0.000000,0.000000,0.713695,0.000000,00:53
6,0.221708,0.228795,0.362947,0.917389,0.842851,0.047801,0.000000,0.000000,0.732591,0.000000,00:53
7,0.200420,0.227566,0.370241,0.918364,0.844149,0.087845,0.000000,0.000000,0.741330,0.000000,00:53
8,0.185899,0.226438,0.370899,0.920642,0.843338,0.082411,0.000067,0.000000,0.749834,0.000000,00:53


epoch,train_loss,valid_loss,miou,background_iou,road_iou,traffic_light_iou,traffic_sign_iou,person_iou,vehicle_iou,bicycle_iou,time
0,0.505617,0.387823,0.303909,0.832589,0.692566,0.000000,0.000000,0.000000,0.602210,0.000000,00:51
1,0.419773,0.369582,0.311889,0.865683,0.726510,0.000000,0.000000,0.000000,0.591029,0.000000,00:51
2,0.348876,0.284566,0.337992,0.897110,0.801128,0.000000,0.000000,0.000000,0.667709,0.000000,00:52
3,0.296886,0.258616,0.346980,0.907943,0.815721,0.000000,0.000000,0.000000,0.705198,0.000000,00:52
4,0.282345,0.303062,0.344276,0.902011,0.818458,0.000000,0.000000,0.000000,0.689461,0.000000,00:53
5,0.248362,0.232243,0.354253,0.913885,0.831946,0.020242,0.000000,0.000000,0.713695,0.000000,00:53
6,0.221708,0.228795,0.362947,0.917389,0.842851,0.047801,0.000000,0.000000,0.732591,0.000000,00:53
7,0.200420,0.227566,0.370241,0.918364,0.844149,0.087845,0.000000,0.000000,0.741330,0.000000,00:53
8,0.185899,0.226438,0.370899,0.920642,0.843338,0.082411,0.000067,0.000000,0.749834,0.000000,00:53


epoch,train_loss,valid_loss,miou,background_iou,road_iou,traffic_light_iou,traffic_sign_iou,person_iou,vehicle_iou,bicycle_iou,time
0,0.505617,0.387823,0.303909,0.832589,0.692566,0.000000,0.000000,0.000000,0.602210,0.000000,00:51
1,0.419773,0.369582,0.311889,0.865683,0.726510,0.000000,0.000000,0.000000,0.591029,0.000000,00:51
2,0.348876,0.284566,0.337992,0.897110,0.801128,0.000000,0.000000,0.000000,0.667709,0.000000,00:52
3,0.296886,0.258616,0.346980,0.907943,0.815721,0.000000,0.000000,0.000000,0.705198,0.000000,00:52
4,0.282345,0.303062,0.344276,0.902011,0.818458,0.000000,0.000000,0.000000,0.689461,0.000000,00:53
5,0.248362,0.232243,0.354253,0.913885,0.831946,0.020242,0.000000,0.000000,0.713695,0.000000,00:53
6,0.221708,0.228795,0.362947,0.917389,0.842851,0.047801,0.000000,0.000000,0.732591,0.000000,00:53
7,0.200420,0.227566,0.370241,0.918364,0.844149,0.087845,0.000000,0.000000,0.741330,0.000000,00:53
8,0.185899,0.226438,0.370899,0.920642,0.843338,0.082411,0.000067,0.000000,0.749834,0.000000,00:53


epoch,train_loss,valid_loss,miou,background_iou,road_iou,traffic_light_iou,traffic_sign_iou,person_iou,vehicle_iou,bicycle_iou,time
0,0.505617,0.387823,0.303909,0.832589,0.692566,0.000000,0.000000,0.000000,0.602210,0.000000,00:51
1,0.419773,0.369582,0.311889,0.865683,0.726510,0.000000,0.000000,0.000000,0.591029,0.000000,00:51
2,0.348876,0.284566,0.337992,0.897110,0.801128,0.000000,0.000000,0.000000,0.667709,0.000000,00:52
3,0.296886,0.258616,0.346980,0.907943,0.815721,0.000000,0.000000,0.000000,0.705198,0.000000,00:52
4,0.282345,0.303062,0.344276,0.902011,0.818458,0.000000,0.000000,0.000000,0.689461,0.000000,00:53
5,0.248362,0.232243,0.354253,0.913885,0.831946,0.020242,0.000000,0.000000,0.713695,0.000000,00:53
6,0.221708,0.228795,0.362947,0.917389,0.842851,0.047801,0.000000,0.000000,0.732591,0.000000,00:53
7,0.200420,0.227566,0.370241,0.918364,0.844149,0.087845,0.000000,0.000000,0.741330,0.000000,00:53
8,0.185899,0.226438,0.370899,0.920642,0.843338,0.082411,0.000067,0.000000,0.749834,0.000000,00:53


epoch,train_loss,valid_loss,miou,background_iou,road_iou,traffic_light_iou,traffic_sign_iou,person_iou,vehicle_iou,bicycle_iou,time
0,0.505617,0.387823,0.303909,0.832589,0.692566,0.000000,0.000000,0.000000,0.602210,0.000000,00:51
1,0.419773,0.369582,0.311889,0.865683,0.726510,0.000000,0.000000,0.000000,0.591029,0.000000,00:51
2,0.348876,0.284566,0.337992,0.897110,0.801128,0.000000,0.000000,0.000000,0.667709,0.000000,00:52
3,0.296886,0.258616,0.346980,0.907943,0.815721,0.000000,0.000000,0.000000,0.705198,0.000000,00:52
4,0.282345,0.303062,0.344276,0.902011,0.818458,0.000000,0.000000,0.000000,0.689461,0.000000,00:53
5,0.248362,0.232243,0.354253,0.913885,0.831946,0.020242,0.000000,0.000000,0.713695,0.000000,00:53
6,0.221708,0.228795,0.362947,0.917389,0.842851,0.047801,0.000000,0.000000,0.732591,0.000000,00:53
7,0.200420,0.227566,0.370241,0.918364,0.844149,0.087845,0.000000,0.000000,0.741330,0.000000,00:53
8,0.185899,0.226438,0.370899,0.920642,0.843338,0.082411,0.000067,0.000000,0.749834,0.000000,00:53
9,0.173698,0.225294,0.376888,0.920903,0.844546,0.122128,0.000869,0.000000,0.749769,0.000000,00:53


In [16]:
samples, outputs, predictions = get_predictions(learn)
table = create_iou_table(samples, outputs, predictions, params.BDD_CLASSES)
wandb.log({"pred_table":table})

In [17]:
scores = learn.validate()
metric_names = ['final_loss'] + [f'final_{x.name}' for x in metrics]
final_results = {metric_names[i] : scores[i] for i in range(len(scores))}
for k,v in final_results.items(): 
    wandb.summary[k] = v

In [18]:
wandb.finish()